In [1]:
import numpy as np
import cv2
from mango_classifier import NaiveMangoClassifier  # assuming you've named it this way
from helper import crop_mango_direct, crop_mango, resize_and_pad, MODEL
from collections import deque, Counter

import contextlib

In [ ]:
%%capture
# Constants
CROP_CONFIDENCE = 0.5

def most_common_prediction(predictions):
    count = Counter(predictions)
    return count.most_common(1)[0][0] if predictions else None

def process_frame(frame, classifier, predictions_queue):
    cv2.imwrite("temp_frame.jpg", frame)
    cropped_img, result_flag = crop_mango(MODEL, "temp_frame.jpg", CROP_CONFIDENCE)
    
    if result_flag == 0:
        final_img = resize_and_pad(cropped_img)
        prediction = classifier.predict_direct(final_img)
        predictions_queue.append(prediction)

        display_prediction = most_common_prediction(predictions_queue)
        color = (0, 255, 0) if display_prediction == "unripe" else (0, 0, 255)
    else:
        display_prediction = most_common_prediction(predictions_queue) or "No mango detected"
        color = (0, 0, 255)

    cv2.putText(frame, f"Prediction: {display_prediction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    return frame

# Initialization
classifier = NaiveMangoClassifier() 
classifier.train('datasets/mango_cut/images/train/ripe', 'datasets/mango_cut/images/train/raw')
input_source = "videos/perfectmango.mp4"
last_20_preds = deque(maxlen=20)

# Video Processing
if input_source.endswith(('.mp4', '.avi', '.gif')):
    cap = cv2.VideoCapture(input_source)
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter('processed_output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        
        processed_frame = process_frame(frame, classifier, last_20_preds)
        out.write(processed_frame)

    cap.release()
    out.release()

# Image Processing
else:
    cropped_img, result_flag = crop_mango_direct(MODEL, input_source, CROP_CONFIDENCE)

    if result_flag == 0:
        final_img = resize_and_pad(cropped_img)
        prediction = classifier.predict_direct(final_img)
        color = (0, 255, 0) if prediction == "unripe" else (0, 0, 255)
        cv2.putText(final_img, f"Prediction: {prediction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.imwrite('processed_image.jpg', final_img)
    else:
        print("No mango detected in the image!")

In [4]:
%%capture
# Now use trained classification model
from model import prediction_output

# Constants
CROP_CONFIDENCE = 0.6

def most_common_prediction(predictions):
    """Returns the most common prediction or None if predictions are empty."""
    if not predictions:
        return None
    count = Counter(predictions)
    return count.most_common(1)[0][0]

def process_frame(frame, predictions_queue):
    """Processes a single video frame or image."""
    cv2.imwrite("temp_frame.jpg", frame)
    cropped_img, result_flag = crop_mango(MODEL, "temp_frame.jpg", CROP_CONFIDENCE)
    
    if result_flag == 0:
        final_img = resize_and_pad(cropped_img)
        prediction = prediction_output(final_img)
        predictions_queue.append(prediction)

        display_prediction = most_common_prediction(predictions_queue)
        color = (0, 255, 0) if display_prediction == "ripe" else (0, 0, 255)
    else:
        display_prediction = most_common_prediction(predictions_queue) or "No mango detected"
        color = (0, 0, 255)

    cv2.putText(frame, f"Prediction: {display_prediction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    return frame

# Initialization
input_source = "videos/perfectmango.mp4"
last_20_preds = deque(maxlen=20)

# Video Processing
if input_source.endswith(('.mp4', '.avi', '.gif')):
    cap = cv2.VideoCapture(input_source)
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter('processed_output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        
        processed_frame = process_frame(frame, last_20_preds)
        out.write(processed_frame)

    cap.release()
    out.release()

# Image Processing
else:
    cropped_img, result_flag = crop_mango_direct(MODEL, input_source, CROP_CONFIDENCE)

    if result_flag == 0:
        final_img = resize_and_pad(cropped_img)
        prediction = prediction_output(final_img)
        color = (0, 255, 0) if prediction == "ripe" else (0, 0, 255)
        cv2.putText(final_img, f"Prediction: {prediction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.imwrite('processed_image.jpg', final_img)
    else:
        print("No mango detected in the image!")


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'

0: 640x384 1 person, 1 orange, 1 potted plant, 107.4ms
Speed: 8.2ms preprocess, 107.4ms inference, 4.2ms postprocess per image at shape (1, 3, 640, 384)


1/1 [==============================] - 0s 318ms/step



0: 640x384 1 person, 1 orange, 1 potted plant, 72.7ms
Speed: 1.7ms preprocess, 72.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)


1/1 [==============================] - 0s 75ms/step



0: 640x384 1 person, 1 orange, 1 potted plant, 84.8ms
Speed: 2.1ms preprocess, 84.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)


...

Skipping due to no detection.


0: 640x384 (no detections), 56.8ms
Speed: 1.7ms preprocess, 56.8ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 51.7ms
Speed: 1.4ms preprocess, 51.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)



Skipping due to no detection.
Skipping due to no detection.


0: 640x384 (no detections), 52.5ms
Speed: 1.3ms preprocess, 52.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)


Skipping due to no detection.
